In [60]:
import pandas as pd
df = pd.read_csv("Log_dataset_combined.csv")
df

,Timestamp,IP_Address,Request_Type,Status_Code,Anomaly_Flag,User_Agent,Session_ID,Location
0,2023-01-01 00:00:00,202.118.116.11,GET,403,0,Edge,4835,Brazil
1,2023-01-01 00:01:00,38.30.40.178,DELETE,301,0,Bot,3176,China
2,2023-01-01 00:02:00,209.5.148.15,POST,500,0,Opera,4312,China
3,2023-01-01 00:03:00,211.116.60.71,GET,301,0,Bot,1003,France
4,2023-01-01 00:04:00,170.166.36.145,POST,404,0,Firefox,1428,Germany
...,...,...,...,...,...,...,...,...
89995,2025-06-06 04:37:50,66.35.22.135,POST,200,0,Edge,2601,Gambia
89996,2025-02-19 17:23:07,130.239.188.233,GET,500,0,Edge,1951,Norway
89997,2025-06-13 05:01:31,14.183.150.102,GET,500,0,Chrome,2640,Gibraltar
89998,2025-07-13 02:34:03,174.160.87.44,POST,404,0,Firefox,2404,Bulgaria


In [ ]:
# Total number of requests
grouped = df.groupby("IP_Address")
total_requests = grouped.size().rename("Total_Requests")

# Request type percentages
req_type = grouped['Request_Type'].value_counts().unstack(fill_value=0)
req_type_perc = req_type.div(req_type.sum(axis=1), axis=0)
req_type_perc.columns = [f"{col}_Perc" for col in req_type_perc.columns]

# Status code classes
def status_class(x):
    if 400 <= x < 500:
        return "4xx"
    elif 500 <= x < 600:
        return "5xx"
    else:
        return "Other"

df['Status_Class'] = df['Status_Code'].apply(status_class)
status_class = df.groupby('IP_Address')['Status_Class'].value_counts().unstack(fill_value=0)
status_class_perc = status_class.div(status_class.sum(axis=1), axis=0)
status_class_perc.columns = [f"{col}_Perc" for col in status_class_perc.columns]


In [62]:
unique_agents = grouped['User_Agent'].nunique().rename("Unique_User_Agents")
unique_sessions = grouped['Session_ID'].nunique().rename("Unique_Sessions")
unique_locations = grouped['Location'].nunique().rename("Unique_Locations")

top_agent = grouped['User_Agent'].agg(lambda x: x.value_counts().idxmax()).rename("Top_User_Agent")
top_location = grouped['Location'].agg(lambda x: x.value_counts().idxmax()).rename("Top_Location")


In [ ]:

ip_level_df = pd.concat([
    total_requests,
    req_type_perc,
    status_class_perc,
    unique_agents,
    unique_sessions,
    unique_locations,
    top_agent,
    top_location,
], axis=1)

ip_level_df.fillna(0, inplace=True)
ip_level_df.reset_index(inplace=True)

# Create anomaly logic
anomaly_ips = (
    (unique_locations > 5) |
    (unique_agents > 3) |
    (status_class.get('4xx_Perc', pd.Series(0)) > 0.25) |
    (status_class.get('5xx_Perc', pd.Series(0)) > 0.25)
)
labels = anomaly_ips.apply(lambda x: -1 if x else 1).rename("Label")
ip_level_df['Label'] = labels




In [65]:
ip_level_df

,index,Total_Requests,DELETE_Perc,GET_Perc,POST_Perc,PUT_Perc,4xx_Perc,5xx_Perc,Other_Perc,Unique_User_Agents,Unique_Sessions,Unique_Locations,Top_User_Agent,Top_Location,Label
0,1.0.25.247,3.0,0.666667,0.000000,0.000000,0.333333,0.333333,0.000000,0.666667,2.0,3.0,3.0,Opera,Mongolia,1
1,1.1.180.252,4.0,0.250000,0.250000,0.500000,0.000000,0.250000,0.250000,0.500000,2.0,4.0,4.0,Firefox,Montenegro,1
2,1.1.21.105,7.0,0.142857,0.285714,0.428571,0.142857,0.428571,0.285714,0.285714,4.0,7.0,7.0,Safari,Australia,-1
3,1.1.62.217,4.0,0.000000,0.500000,0.250000,0.250000,0.000000,0.750000,0.250000,3.0,4.0,4.0,Chrome,Switzerland,1
4,1.100.140.68,4.0,0.000000,0.500000,0.500000,0.000000,0.500000,0.000000,0.500000,3.0,4.0,4.0,Bot,Belize,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33975,99.96.176.36,5.0,0.400000,0.200000,0.400000,0.000000,0.200000,0.200000,0.600000,3.0,5.0,5.0,Firefox,Libyan Arab Jamahiriya,1
33976,99.97.118.185,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0,1.0,Bot,Antarctica (the territory South of 60 deg S),1
33977,99.97.154.134,1.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.0,1.0,1.0,Safari,USA,1
33978,99.99.53.56,6.0,0.333333,0.166667,0.166667,0.333333,0.333333,0.000000,0.666667,4.0,6.0,6.0,Safari,French Guiana,-1


In [66]:
ip_level_df['Label'].value_counts()


Label
 1    28591
-1     5389
Name: count, dtype: int64

In [67]:
ip_level_df.to_csv("IP_Level_log_dataset_labeled.csv", index=False)
